# From previous night (#6) - target profile we've found:
```py                                                    
customerid name         phone        birthdate  address          citystatezip     Margin (*)
4167       Sherri Long  585-838-9161 1975-04-09 2092 Seward Ave  Bronx, NY 10473  -92.03
```

In [1]:
import pandas as pd
import os
filenames = [_.split('.')[0] for _ in os.listdir('./') if '.csv' in _]
print(filenames)

dfs = [pd.read_csv(filename + '.csv') for filename in filenames]
df_orders, df_prod, df_order_items, df_customers = dfs
print([len(_) for _ in [df_orders, df_prod, df_order_items, df_customers]])

['noahs-orders', 'noahs-products', 'noahs-orders_items', 'noahs-customers']
[213232, 1278, 426541, 8260]


# Step 1 - Get **Sherri** at *4167*

In [2]:
cousin_night_6 = df_customers.set_index("customerid").loc[[4167]].reset_index()
cousin_night_6

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long
0,4167,Sherri Long,2092 Seward Ave,"Bronx, NY 10473",1975-04-09,585-838-9161,America/New_York,40.8182,-73.86752


# Step 2 - items w/o extra descriptions
- ### (1/2) prod[ desc ] : sample

In [3]:
print('/regular [desc] section looks like this')
df_prod.head()

/regular [desc] section looks like this


,sku,desc,wholesale_cost,dims_cm
0,PET0002,"Wet Cat Food, Tuna & Tuna",0.89,16.1|5.5|3.2
1,PET0006,"Vegan Adult Cat Food, Chicken & Chicken",1.41,19.9|16.9|0.3
2,PET0019,"Dry Senior Cat Food, Tuna & Tuna",0.89,18.6|8.7|7.2
3,PET0023,"Gluten-free Cat Food, Beef & Beef",1.47,17.4|4.4|4.0
4,PET0027,"Wet Adult Cat Food, Tuna & Tuna",1.03,14.3|14.3|3.0


- ### (2/2) prod[ desc ] : parenthesis-included items 

In [4]:
pattern_extra_desc = "\\(.*\\)" # extra desc comes inside parenthesis
df_prod[
    df_prod['desc'].str.contains(pattern_extra_desc, regex=True)
]

,sku,desc,wholesale_cost,dims_cm
5,COL0037,Noah's Jewelry (green),28.32,17.4|11.2|5.7
6,COL0041,Noah's Ark Model (HO Scale),2487.35,7.2|4.3|0.4
8,COL0065,Noah's Jewelry (mauve),33.52,19.0|12.2|10.5
21,COL0166,Noah's Action Figure (blue),13.98,12.1|7.7|7.2
22,COL0167,Noah's Bobblehead (blue),5.36,8.9|5.6|0.6
...,...,...,...,...
1202,COL9420,Noah's Jewelry (amber),30.01,13.8|9.1|4.9
1246,COL9773,Noah's Poster (magenta),4.13,7.2|4.9|1.9
1253,COL9819,Noah's Lunchbox (blue),9.87,14.2|4.1|3.3
1265,TOY9896,Jigsaw Puzzle (Noah's Ark),1.42,14.2|11.1|10.4


- ### (3/2) prod[ desc ] : create a no-parenthesis column

In [5]:
df_prod['desc_without_parenthesis'] = df_prod['desc'].str.replace("\\(.*\\)", "", regex=True)
df_prod.head()#['desc_without_parenthesis']

,sku,desc,wholesale_cost,dims_cm,desc_without_parenthesis
0,PET0002,"Wet Cat Food, Tuna & Tuna",0.89,16.1|5.5|3.2,"Wet Cat Food, Tuna & Tuna"
1,PET0006,"Vegan Adult Cat Food, Chicken & Chicken",1.41,19.9|16.9|0.3,"Vegan Adult Cat Food, Chicken & Chicken"
2,PET0019,"Dry Senior Cat Food, Tuna & Tuna",0.89,18.6|8.7|7.2,"Dry Senior Cat Food, Tuna & Tuna"
3,PET0023,"Gluten-free Cat Food, Beef & Beef",1.47,17.4|4.4|4.0,"Gluten-free Cat Food, Beef & Beef"
4,PET0027,"Wet Adult Cat Food, Tuna & Tuna",1.03,14.3|14.3|3.0,"Wet Adult Cat Food, Tuna & Tuna"


# Step 3 - dates w/o specific hours

In [6]:
df_orders['date_without_hours'] = pd.to_datetime( df_orders['ordered'] ).dt.strftime('%d_%m_%Y')
df_orders.head()#["date_without_hours"]

,orderid,customerid,ordered,shipped,items,total,date_without_hours
0,1001,6878,2017-01-31 02:56:45,2017-01-31 09:00:00,NaN,0.99,31_01_2017
1,1002,6375,2017-01-31 04:13:35,2017-01-31 12:15:00,NaN,13.59,31_01_2017
2,1003,8045,2017-01-31 04:45:12,2017-01-31 10:45:00,NaN,1.23,31_01_2017
3,1004,5385,2017-01-31 05:49:19,2017-01-31 09:00:00,NaN,2.10,31_01_2017
4,1005,9235,2017-01-31 06:28:11,2017-01-31 16:00:00,NaN,13.23,31_01_2017


# Step 4 - we want to basically _do_ :
```py
SELECT *
FROM customers
JOIN orders ON customers.customerid = orders.customerid
JOIN order_items ON orders.orderid = order_items.orderid
JOIN products ON order_items.sku = products.sku;
```

In [7]:
"""
fully_denormalized = (
    df_customers.merge(df_orders, on="customerid")
        .merge(df_order_items, on="orderid")
        .merge(df_prod, on="sku")
)
"""
temp = df_customers.merge(df_orders, on="customerid")
temp.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,orderid,ordered,shipped,items,total,date_without_hours
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,1509,2017-02-05 10:16:57,2017-02-05 14:15:00,NaN,13.22,05_02_2017
1,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,3906,2017-02-28 07:11:18,2017-02-28 18:00:00,NaN,14.40,28_02_2017
2,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,4449,2017-03-05 12:53:30,2017-03-05 12:53:30,NaN,82.70,05_03_2017
3,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,4556,2017-03-06 13:30:10,2017-03-06 13:30:10,NaN,3.22,06_03_2017
4,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,4924,2017-03-10 10:08:17,2017-03-10 12:45:00,NaN,1.34,10_03_2017


In [8]:
temp = temp.merge(df_order_items, on="orderid")
temp.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,orderid,ordered,shipped,items,total,date_without_hours,sku,qty,unit_price
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,1509,2017-02-05 10:16:57,2017-02-05 14:15:00,NaN,13.22,05_02_2017,DLI8752,1,1.25
1,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,1509,2017-02-05 10:16:57,2017-02-05 14:15:00,NaN,13.22,05_02_2017,PET3601,1,1.19
2,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,1509,2017-02-05 10:16:57,2017-02-05 14:15:00,NaN,13.22,05_02_2017,TOY8287,1,10.78
3,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,3906,2017-02-28 07:11:18,2017-02-28 18:00:00,NaN,14.40,28_02_2017,COL6858,1,14.40
4,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,4449,2017-03-05 12:53:30,2017-03-05 12:53:30,NaN,82.70,05_03_2017,PET9025,1,1.83


In [9]:
temp = temp.merge(df_prod, on="sku")
temp.head()

,customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long,orderid,...,items,total,date_without_hours,sku,qty,unit_price,desc,wholesale_cost,dims_cm,desc_without_parenthesis
0,1001,Jacqueline Alvarez,105N Elizabeth St,"Manhattan, NY 10013",1958-01-23,315-377-5031,America/New_York,40.71817,-73.997468,1509,...,NaN,13.22,05_02_2017,DLI8752,1,1.25,Smoked Cucumber Salad,0.95,15.6|14.2|2.9,Smoked Cucumber Salad
1,1011,Morgan Robinson,448 E 81st St,"Manhattan, NY 10028",1951-03-09,917-466-5214,America/New_York,40.77575,-73.957890,174869,...,NaN,3.54,15_11_2021,DLI8752,2,1.17,Smoked Cucumber Salad,0.95,15.6|14.2|2.9,Smoked Cucumber Salad
2,1033,Kent Nunez,120 Richardson St,"Brooklyn, NY 11211",1985-05-05,914-572-1691,America/New_York,40.71855,-73.943120,159191,...,NaN,27.51,08_06_2021,DLI8752,1,1.19,Smoked Cucumber Salad,0.95,15.6|14.2|2.9,Smoked Cucumber Salad
3,1060,Christopher Garcia,16½ W 75th St,"Manhattan, NY 10023",1942-04-30,716-263-0817,America/New_York,40.78053,-73.982910,70467,...,NaN,1.31,31_12_2018,DLI8752,1,1.31,Smoked Cucumber Salad,0.95,15.6|14.2|2.9,Smoked Cucumber Salad
4,1078,Thomas Garcia,141 Branton St,"Brooklyn, NY 11236",1961-03-15,716-268-5311,America/New_York,40.64549,-73.919010,152090,...,NaN,4.82,29_03_2021,DLI8752,1,1.17,Smoked Cucumber Salad,0.95,15.6|14.2|2.9,Smoked Cucumber Salad


In [10]:
same_prods = cousin_night_6.merge(temp, on='customerid')[["desc", "desc_without_parenthesis", "date_without_hours"]]
same_prods[ same_prods['desc'] != same_prods['desc_without_parenthesis']]

,desc,desc_without_parenthesis,date_without_hours
0,Noah's Jersey (blue),Noah's Jersey,07_10_2021
3,Noah's Poster (white),Noah's Poster,23_04_2022
52,Noah's Poster (azure),Noah's Poster,31_12_2018
53,Noah's Action Figure (green),Noah's Action Figure,28_06_2020


In [11]:
res = same_prods.merge(temp, on=['desc_without_parenthesis','date_without_hours'])
res[ res['desc_x'] != res['desc_y'] ]

,desc_x,desc_without_parenthesis,date_without_hours,customerid,name,address,citystatezip,birthdate,phone,timezone,...,ordered,shipped,items,total,sku,qty,unit_price,desc_y,wholesale_cost,dims_cm
6,Noah's Poster (white),Noah's Poster,23_04_2022,1162,Jeffrey Johnson,385½ E 74th St,"Manhattan, NY 10021",1984-08-06,516-810-7590,America/New_York,...,2022-04-23 15:31:27,2022-04-23 15:31:27,NaN,11.86,COL7063,1,4.71,Noah's Poster (azure),4.11,17.8|15.2|1.9
90,Noah's Poster (azure),Noah's Poster,31_12_2018,7698,Erika Mcconnell,8579 21st Ave,"Brooklyn, NY 11214",1956-11-17,315-248-5742,America/New_York,...,2018-12-31 08:38:36,2018-12-31 11:45:00,NaN,7.85,COL0837,1,4.57,Noah's Poster (mauve),3.86,13.0|3.3|0.8
91,Noah's Poster (azure),Noah's Poster,31_12_2018,2610,Karen Contreras,169 Allen St,"Manhattan, NY 10002",1993-10-28,838-578-7266,America/New_York,...,2018-12-31 08:54:05,2018-12-31 15:30:00,NaN,11.82,COL5018,1,6.17,Noah's Poster (puce),4.41,19.1|17.8|13.6
92,Noah's Poster (azure),Noah's Poster,31_12_2018,5783,Carlos Myers,1486 Bath Ave,"Brooklyn, NY 11228",1986-04-27,838-335-7157,America/New_York,...,2018-12-31 12:26:39,2018-12-31 12:26:39,NaN,4.57,COL2467,1,4.57,Noah's Poster (orange),3.58,16.3|6.1|4.7
93,Noah's Poster (azure),Noah's Poster,31_12_2018,7474,Alex Evans,110B Warren St,"Manhattan, NY 10007",1983-08-02,914-514-2194,America/New_York,...,2018-12-31 13:02:52,2018-12-31 13:02:52,NaN,4.27,COL2467,1,4.27,Noah's Poster (orange),3.58,16.3|6.1|4.7
95,Noah's Action Figure (green),Noah's Action Figure,28_06_2020,6045,Sheila Lopez,592 Throop Ave,"Brooklyn, NY 11216",1963-01-06,516-847-5723,America/New_York,...,2020-06-28 14:59:03,2020-06-28 16:45:00,NaN,56.15,COL3038,1,16.12,Noah's Action Figure (purple),13.59,14.1|11.9|11.2


### guessed: Carlos Myers